# Module 03

## Session 12 Recommendation System

# Content Based Filtering

# Exercise

# Data & Function: single user

In [1]:
import pandas as pd
import numpy as np

movies = ["Terminator 2","Interstellar","Ant Man 2","3 Idiots"]
scores = [7,9,8,9]
action = [1,0,1,0]
scifi = [1,1,1,0]
adventure = [0,1,1,0]
comedy = [0,0,1,1]
drama = [0,1,0,1]

df_movies = pd.DataFrame({
    'movie':movies,
    'scores':scores,
    'Action':action,
    'Sci-Fi':scifi,
    'Adventure':adventure,
    'Comedy':comedy,
    'Drama':drama
})

movies = ["Titanic",'Martian','GOTG Vol 2']
action = [1,0,1]
scifi = [1,1,1]
adventure = [0,1,1]
comedy = [0,0,1]
drama = [0,1,0]

df_movies_recommendation = pd.DataFrame({
    'movie':movies,
    'Action':action,
    'Sci-Fi':scifi,
    'Adventure':adventure,
    'Comedy':comedy,
    'Drama':drama
})

def user_recommendation(df_movies,df_movies_recommendation):
  df_movies2 = df_movies.copy()
  df_movies2.drop('movie', axis = 1, inplace = True)

  for i in df_movies.columns[2:]:
    df_movies2[i] = df_movies2['scores']*df_movies2[i] 

  df_movies2.drop('scores', axis = 1, inplace = True)
  movie_scoring = df_movies2.sum()/df_movies2.sum().sum()

  for i in df_movies.columns[2:]:
    df_movies_recommendation[i] = df_movies_recommendation[i]*movie_scoring[i]

  df_movies_recommendation['movie rating prediction'] = df_movies_recommendation.sum(axis = 1)
  return df_movies_recommendation[['movie','movie rating prediction']]

In [2]:
df_movies

,movie,scores,Action,Sci-Fi,Adventure,Comedy,Drama
0,Terminator 2,7,1,1,0,0,0
1,Interstellar,9,0,1,1,0,1
2,Ant Man 2,8,1,1,1,1,0
3,3 Idiots,9,0,0,0,1,1


In [3]:
df_movies_recommendation

,movie,Action,Sci-Fi,Adventure,Comedy,Drama
0,Titanic,1,1,0,0,0
1,Martian,0,1,1,0,1
2,GOTG Vol 2,1,1,1,1,0


In [4]:
user_recommendation(df_movies, df_movies_recommendation)

/var/folders/cn/4ghsh5pd05g7zlzsc2zdmx000000gn/T/ipykernel_2082/1297389254.py:51: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_movies_recommendation['movie rating prediction'] = df_movies_recommendation.sum(axis = 1)


,movie,movie rating prediction
0,Titanic,0.428571
1,Martian,0.648352
2,GOTG Vol 2,0.802198


# Data & Function: multiple user

In [5]:
movies = ["Terminator 2","Interstellar","Ant Man 2","3 Idiots"]
action = [1,0,1,0]
scifi = [1,1,1,0]
adventure = [0,1,1,0]
comedy = [0,0,1,1]
drama = [0,1,0,1]

df_item_features = pd.DataFrame({
    'movie':movies,
    'Action':action,
    'Sci-Fi':scifi,
    'Adventure':adventure,
    'Comedy':comedy,
    'Drama':drama
})

user = ['user 1','user 2','user 3','user 4']
terminator_2 = [7,8,9,0]
interstellar = [9,0,0,7]
ant_man_2 = [8,6,0,0]
three_idiots = [9,5,10,9]

df_user_items = pd.DataFrame({
    'user':user,
    'Terminator 2':terminator_2,
    'Interstellar':interstellar,
    'Ant Man 2':ant_man_2,
    '3 Idiots':three_idiots
})

movies = ["Titanic","Martian","GOTG Vol 2"]
action = [1,0,1]
scifi = [1,1,1]
adventure = [0,1,1]
comedy = [0,0,1]
drama = [0,1,0]

df_item_features_new = pd.DataFrame({
    'movie':movies,
    'Action':action,
    'Sci-Fi':scifi,
    'Adventure':adventure,
    'Comedy':comedy,
    'Drama':drama
})


def user_recommendation_multiple(df_user_items,df_item_features,df_item_features_new):
  
  arr_user_items = np.array(df_user_items.drop('user', axis = 1))
  arr_item_features = np.array(df_item_features.drop('movie', axis = 1))

  n_user = arr_user_items.shape[0]
  n_item = arr_user_items.shape[1]
  n_feature = arr_item_features.shape[1]

  arr_user_items_score = np.empty((n_user,n_item))
  arr_user_feature = np.empty((n_user,n_feature))

  for i in range(0,n_user):
    # print(arr_user_items[i,:])
    user_feature = np.matmul(arr_user_items[i,:],arr_item_features)
    # print(user_feature)
    user_feature = user_feature/user_feature.sum()
    arr_user_feature[i,:] = user_feature

  df_user_feature = pd.DataFrame(arr_user_feature)
  df_user_feature.columns = df_item_features.columns[1:]
  df_user_feature.index = user

  for i in range(0, n_user):
    user_item_score = np.matmul(arr_item_features,arr_user_feature[i,:])
    arr_user_items_score[i,:] = user_item_score

  arr_user_items_score_unwatched = np.where(arr_user_items == 0,arr_user_items_score,0)

  df_user_items_score_unwatched = pd.DataFrame(arr_user_items_score_unwatched)
  df_user_items_score_unwatched.columns = df_user_items.columns[1:]
  df_user_items_score_unwatched.index = df_user_items['user']

  arr_item_features_new = np.array(df_item_features_new.drop('movie', axis = 1))

  n_item_new = df_item_features_new.shape[0]

  arr_user_items_score_new = np.empty((n_user,n_item_new))

  for i in range(0, n_user):
    user_item_score = np.matmul(arr_item_features_new,arr_user_feature[i,:])
    arr_user_items_score_new[i,:] = user_item_score

  df_user_items_score_new = pd.DataFrame(arr_user_items_score_new)
  df_user_items_score_new.index = user
  df_user_items_score_new.columns = df_item_features_new['movie']

  return [df_user_items_score_unwatched,df_user_items_score_new]

In [6]:
df_item_features

,movie,Action,Sci-Fi,Adventure,Comedy,Drama
0,Terminator 2,1,1,0,0,0
1,Interstellar,0,1,1,0,1
2,Ant Man 2,1,1,1,1,0
3,3 Idiots,0,0,0,1,1


In [7]:
df_user_items

,user,Terminator 2,Interstellar,Ant Man 2,3 Idiots
0,user 1,7,9,8,9
1,user 2,8,0,6,5
2,user 3,9,0,0,10
3,user 4,0,7,0,9


In [8]:
df_item_features_new

,movie,Action,Sci-Fi,Adventure,Comedy,Drama
0,Titanic,1,1,0,0,0
1,Martian,0,1,1,0,1
2,GOTG Vol 2,1,1,1,1,0


## Objective:
1. Unwatch movie recommendation
2. Recommendation for new movie

In [12]:
old_movie_recommendation, new_movie_recommendation = user_recommendation_multiple(df_user_items, df_item_features, df_item_features_new)

In [13]:
old_movie_recommendation

,Terminator 2,Interstellar,Ant Man 2,3 Idiots
user,,,,
user 1,0.000000,0.0,0.000000,0.0
user 2,0.000000,0.5,0.000000,0.0
user 3,0.000000,0.5,0.736842,0.0
user 4,0.179487,0.0,0.589744,0.0


In [14]:
new_movie_recommendation

movie,Titanic,Martian,GOTG Vol 2
user 1,0.428571,0.648352,0.802198
user 2,0.560000,0.500000,0.900000
user 3,0.473684,0.500000,0.736842
user 4,0.179487,0.769231,0.589744


Recommendation:
* user 1: GOTG vol2, martian
* user 2: GOTG vol2
* user 3: GOTG vol2
* user 4: martian